In [ ]:
%pip install pandas numpy scikit-learn pickle

ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle


In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [3]:
data = pd.read_csv('top_50_indian_destinations.csv')
data.head()

,destination_id,name,culture_score,nature_score,adventure_score,relaxation_score,spiritual_score,food_score
0,1,Agra,7,3,4,6,5,8
1,2,Jaipur,9,5,6,7,6,9
2,3,Varanasi,8,4,5,6,10,7
3,4,Goa,6,9,7,10,5,9
4,5,Manali,5,10,9,8,4,7


In [4]:
# Select relevant features
features = ["culture_score", "nature_score", "adventure_score", "relaxation_score", "spiritual_score", "food_score"]

# Create feature matrix and normalize
features_matrix = data[features].values
features_matrix = features_matrix / np.max(features_matrix, axis=0)

# Show normalized feature matrix
features_matrix[:5]

array([[0.77777778, 0.3       , 0.4       , 0.6       , 0.5       ,
        0.8       ],
       [1.        , 0.5       , 0.6       , 0.7       , 0.6       ,
        0.9       ],
       [0.88888889, 0.4       , 0.5       , 0.6       , 1.        ,
        0.7       ],
       [0.66666667, 0.9       , 0.7       , 1.        , 0.5       ,
        0.9       ],
       [0.55555556, 1.        , 0.9       , 0.8       , 0.4       ,
        0.7       ]])

In [ ]:
def get_similar_destinations(destination_id, top_n=5):
    """
    Find destinations similar to the given destination.
    """
    idx = data[data['destination_id'] == destination_id].index[0]

    # Compute similarity scores
    destination_features = features_matrix[idx].reshape(1, -1)
    similarity_scores = cosine_similarity(destination_features, features_matrix)[0]

    # Get top similar destinations (excluding self)
    similar_indices = np.argsort(similarity_scores)[::-1][1:top_n+1]
    similar_destinations = data.iloc[similar_indices].copy()
    similar_destinations['similarity_score'] = similarity_scores[similar_indices]

    return similar_destinations

In [7]:
def recommend_by_preferences(preferences, top_n=5):
    """
    Get recommendations based on user preferences.
    """
    pref_vector = np.array([
        preferences.get("culture", 5),
        preferences.get("nature", 5),
        preferences.get("adventure", 5),
        preferences.get("relaxation", 5),
        preferences.get("spiritual", 5),
        preferences.get("food", 5)
    ]).reshape(1, -1) / 10  # Normalize to 0-1 scale

    # Compute similarity scores
    similarity_scores = cosine_similarity(pref_vector, features_matrix)[0]

    # Get top recommendations
    top_indices = np.argsort(similarity_scores)[::-1][:top_n]
    recommendations = data.iloc[top_indices].copy()
    recommendations["match_score"] = (similarity_scores[top_indices] * 100).round(1)

    return recommendations

In [8]:
# Save the feature matrix and dataset
with open("model2.pkl", "wb") as f:
    pickle.dump((data, features_matrix), f)

print("Model saved successfully!")


Model saved successfully!
